Cryptocurrency trading bot.

In [1]:
##%pip install python-binance==0.7.5
#%pip install --upgrade python-binance
#%pip install paramiko

## API key management.

In [2]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
client

## Get all available pairs for information needed to trade.

In [3]:
from cryptocurrency.exchange import Cryptocurrency_exchange
exchange = Cryptocurrency_exchange(client=client, directory='crypto_logs')
exchange_info = exchange.info
exchange_info

,symbol,base_asset,base_asset_precision,quote_asset,quote_precision,quote_asset_precision,base_asset_commission,quote_commission_precision,allow_trailing_stop,cancel_replace_allowed,min_price,max_price,tick_size,step_size,multiplier_up,multiplier_down
0,ETHBTC,ETH,8,BTC,8,8,8,8,True,True,1.000000e-06,922327.0,1.000000e-06,0.0001,5.0,0.2
1,LTCBTC,LTC,8,BTC,8,8,8,8,True,True,1.000000e-06,100000.0,1.000000e-06,0.0010,5.0,0.2
2,BNBBTC,BNB,8,BTC,8,8,8,8,True,True,1.000000e-06,100000.0,1.000000e-06,0.0010,5.0,0.2
3,NEOBTC,NEO,8,BTC,8,8,8,8,True,True,1.000000e-06,100000.0,1.000000e-06,0.0100,5.0,0.2
4,QTUMETH,QTUM,8,ETH,8,8,8,8,True,True,1.000000e-06,1000.0,1.000000e-06,0.1000,5.0,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1426,OSMOUSDT,OSMO,8,USDT,8,8,8,8,True,True,1.000000e-03,1000.0,1.000000e-03,0.0100,5.0,0.2
1427,OSMOBUSD,OSMO,8,BUSD,8,8,8,8,True,True,1.000000e-03,1000.0,1.000000e-03,0.0100,5.0,0.2
1428,HFTBTC,HFT,8,BTC,8,8,8,8,True,True,1.000000e-08,1.0,1.000000e-08,0.1000,5.0,0.2
1429,HFTBUSD,HFT,8,BUSD,8,8,8,8,True,True,1.000000e-04,1000.0,1.000000e-04,0.1000,5.0,0.2


## Get conversion table for price conversions.

In [4]:
from_asset = 'BDOT'
to_asset = 'BETH'

from cryptocurrency.conversion import get_shortest_pair_path_between_assets

shortest_pair_path_between_assets = \
    get_shortest_pair_path_between_assets(from_asset, to_asset, 
                                          exchange_info)

shortest_pair_path_between_assets

[('BDOT', 'DOT'), ('DOT', 'ETH'), ('BETH', 'ETH')]

In [5]:
from cryptocurrency.conversion_table import get_conversion_table
conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
conversion_table

,symbol,base_asset,quote_asset,is_shorted,price_change_percent,weighted_average_price,open,high,low,close,...,rolling_traded_volume,traded_bid_volume,traded_ask_volume,traded_price,traded_bid_price,traded_ask_price,bid_ask_change_percent,bid_ask_volume_percent,traded_bid_ask_change_percent,traded_bid_ask_volume_percent
0,BUSDVAI,BUSD,VAI,False,0.770,1.02035400,1.039000,1.051000,1.011000,1.047000,...,1.425284e+10,9.280936e+08,5.070173e+07,490.568230,NaN,NaN,0.191022,0.297240,NaN,94.819987
1,USDTIDRT,USDT,IDRT,False,-0.925,15761.60,15887.000000,15925.000000,15594.000000,15740.000000,...,2.338486e+10,1.112708e+08,1.992097e+08,628.238763,NaN,NaN,0.025413,71.962617,NaN,35.838244
2,OSMOBTC,OSMO,BTC,False,7.091,0.00007209,0.000068,0.000078,0.000068,0.000073,...,2.206288e+06,2.360043e+03,1.438976e+03,1.286000,1.467838,1.474801,0.476580,2.975027,0.472152,62.122433
3,PONDBUSD,POND,BUSD,False,18.170,0.00856215,0.007760,0.009450,0.007710,0.009170,...,2.250545e+06,1.410274e+01,6.665382e+00,0.009200,0.000067,0.000067,0.108932,21.528909,0.000000,67.905709
4,TROYBUSD,TROY,BUSD,False,11.335,0.00273062,0.002585,0.003003,0.002581,0.002878,...,2.862778e+07,9.497224e+00,2.536336e+00,0.002876,0.000008,0.000008,0.243563,14.241849,0.000000,78.922813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1426,BTCUSDT,BTC,USDT,False,8.659,16993.14810049,16196.230000,18199.000000,15919.040000,17598.630000,...,1.706009e+10,4.709156e+10,1.170719e+11,16584.555989,NaN,NaN,0.004716,20.251716,NaN,28.685776
1427,BTCBUSD,BTC,BUSD,False,8.585,16974.63372686,16162.460000,18169.000000,15886.960000,17550.030000,...,1.706009e+10,4.709156e+10,1.170719e+11,16584.555989,NaN,NaN,0.009857,90.223136,NaN,28.685776
1428,DOGEGBP,DOGE,GBP,False,13.185,0.07361077,0.067120,0.080530,0.065320,0.075970,...,6.928923e+08,6.522576e+04,2.216124e+06,82.380954,inf,inf,0.144718,7.296606,NaN,2.859086
1429,BUSDUSDT,BUSD,USDT,False,0.060,1.00661566,1.002200,1.032000,1.001300,1.002800,...,1.425284e+10,9.280936e+08,5.070173e+07,490.568230,NaN,NaN,0.009972,77.012563,NaN,94.819987


## Pre-screen buyable base assets in pairs.

In [6]:
from cryptocurrency.conversion_table import get_new_tickers
tickers_list = get_new_tickers(conversion_table=conversion_table)
len(tickers_list)

1431

## Locally (saving time) convert price from an asset to another using existing pairs.

In [7]:
from cryptocurrency.trade import select_asset_with_biggest_wallet
from_asset, converted_quantity, quantity = \
    select_asset_with_biggest_wallet(client=client, 
                                     conversion_table=conversion_table, 
                                     exchange_info=exchange_info, 
                                     as_pair=True)
from_asset, converted_quantity, quantity

('BTC', '10.26', 0.00058356)

## Trade.

In [8]:
import pandas as pd

def get_conversion_table_2(conversion_table_file_name='crypto_logs/conversion_table.txt'):
    return pd.read_csv(conversion_table_file_name, index_col=0)

conversion_table_file_name = 'crypto_logs/conversion_table.txt'
conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, 
                                        as_pair=True, dump_raw=True)
conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
conversion_table

,symbol,price_change,price_change_percent,weighted_average_price,close_shifted,close,last_volume,bid_price,bid_volume,ask_price,...,low,rolling_base_volume,rolling_quote_volume,open_time,close_time,first_ID,last_ID,count,base_asset,quote_asset
0,ETHBTC,0.004565,6.579,0.072698,0.069387,0.073953,1.1058,0.073953,3.0473,0.073954,...,0.069368,2.865194e+05,2.082949e+04,1668039314506,1668125714506,388294675,389021757,727083,ETH,BTC
1,LTCBTC,0.000225,7.003,0.003311,0.003211,0.003438,4.2680,0.003438,3.7920,0.003439,...,0.003186,4.156939e+05,1.376514e+03,1668039314550,1668125714550,85069512,85179612,110101,LTC,BTC
2,BNBBTC,0.000498,2.979,0.017027,0.016723,0.017217,1.1240,0.017217,4.2380,0.017218,...,0.016517,2.396858e+05,4.081155e+03,1668039314466,1668125714466,209106296,209432157,325862,BNB,BTC
3,NEOBTC,0.000016,4.113,0.000403,0.000388,0.000405,31.6500,0.000405,85.4800,0.000406,...,0.000386,1.244883e+05,5.017460e+01,1668039314043,1668125714043,44003464,44008793,5330,NEO,BTC
4,QTUMETH,-0.000026,-1.440,0.001793,0.001826,0.001780,17.8000,0.001779,216.8000,0.001783,...,0.001711,1.566240e+04,2.808910e+01,1668039314266,1668125714266,5339196,5339600,405,QTUM,ETH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,OSMOUSDT,0.170000,15.260,1.221778,1.115000,1.284000,25.7700,1.284000,362.1000,1.289000,...,1.104000,8.310286e+05,1.015332e+06,1668039312378,1668125712378,371835,381217,9383,OSMO,USDT
2122,OSMOBUSD,0.175000,15.752,1.219456,1.113000,1.286000,802.9400,1.280000,1046.3800,1.286000,...,1.102000,6.922431e+05,8.441600e+05,1668039311482,1668125711482,398985,404622,5638,OSMO,BUSD
2123,HFTBTC,-0.000001,-3.802,0.000039,0.000038,0.000037,251.3000,0.000037,13.7000,0.000037,...,0.000034,5.845108e+06,2.265947e+02,1668039314332,1668125714332,166771,183022,16252,HFT,BTC
2124,HFTBUSD,0.030400,4.927,0.654892,0.617400,0.647400,863.3000,0.647400,516.6000,0.648500,...,0.561300,5.621146e+07,3.681243e+07,1668039314168,1668125714168,928251,1038200,109950,HFT,BUSD


In [9]:
to_asset = 'BUSD'

from cryptocurrency.trade import trade
import pandas as pd

def get_conversion_table_2(conversion_table_file_name='crypto_logs/conversion_table.txt'):
    return pd.read_csv(conversion_table_file_name, index_col=0)

conversion_table_file_name = 'crypto_logs/conversion_table.txt'
conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                exchange_info=exchange_info)
request

[('BTC', 'BUSD')]


{'symbol': 'BTCBUSD',
 'orderId': 6996332738,
 'orderListId': -1,
 'clientOrderId': '9W2Fs9xEg7OKYaJDnoPuAL',
 'transactTime': 1668125788644,
 'price': '0.00000000',
 'origQty': '0.00058000',
 'executedQty': '0.00058000',
 'cummulativeQuoteQty': '10.17490520',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'fills': [{'price': '17542.94000000',
   'qty': '0.00058000',
   'commission': '0.00000000',
   'commissionAsset': 'BNB',
   'tradeId': 611249440}]}

## Main loop buying screened assets in pairs which are connected to the wallet holdings.

In [ ]:
sell_asset = 'BTC'
keys_file = 'contabo_keys.txt'
#crypto_output_log_screened = '~/workspace/crypto_logs/crypto_output_log_15s_screened.txt'
#crypto_output_log_screened = '~/workspace/crypto_logs/crypto_output_log_1min_screened.txt'
crypto_output_log_screened = '~/workspace/crypto_logs/crypto_output_log_30min_screened.txt'

with open(keys_file, 'r') as f:
    password = f.readline().replace('\n', '')

import sys
import time
import pandas as pd
import paramiko

if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect('154.12.239.24', username='sam', port=22, password=password)

while True:
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command('cat {}'.format(crypto_output_log_screened))

    tradable_pairs = pd.read_csv(StringIO(str(ssh_stdout.read().decode("utf-8"))))

    if tradable_pairs.shape[0] < 1:
        #print('None')
        to_asset = sell_asset
    else:
        tradable_assets = tradable_pairs['symbol']
        tradable_assets = list(set(tradable_assets.tolist()))
        #print(tradable_assets)
        print('.', end='')
        latest_asset = tradable_assets[-1]
        if from_asset not in tradable_assets:
            to_asset = latest_asset

    if from_asset != to_asset:
        conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
        from_asset, converted_quantity, quantity = \
            select_asset_with_biggest_wallet(client=client, conversion_table=conversion_table, 
                                             exchange_info=exchange_info)
        request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                        exchange_info=exchange_info)
        if request is not None:
            if to_asset != sell_asset:
                pair_bought = request['symbol']
                price_when_bought = float(request['fills'][0]['price'])
            else:
                asset_to_delete = from_asset
            from_asset = to_asset

    time.sleep(5)

ssh.close()

.[('BAND', 'BUSD')]
......

In [ ]:
take_profit = None
stop_loss = None
sell_asset = 'BTC'
bought_asset = None
asset_to_delete = None
keys_file = 'contabo_keys.txt'
crypto_input_log_screened = 'crypto_logs/crypto_input_log_15s_screened.txt'
crypto_output_log_screened = '~/workspace/crypto_logs/crypto_output_log_1min_screened.txt'

with open(keys_file, 'r') as f:
    password = f.readline().replace('\n', '')

from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.conversion import get_base_asset_from_pair
from cryptocurrency.exchange import Cryptocurrency_exchange

import sys
import time
import pandas as pd
import paramiko

if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect('154.12.239.24', username='sam', port=22, password=password)

client = Cryptocurrency_authenticator(use_keys=False, testnet=False).spot_client
exchange_info = Cryptocurrency_exchange(client=client, directory='crypto_logs').info

conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
#conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
from_asset, converted_quantity, quantity = \
    select_asset_with_biggest_wallet(client=client, conversion_table=conversion_table, 
                                     exchange_info=exchange_info)

while True:
    try:
        if asset_to_delete is None:
            #tradable_assets = pd.read_csv(crypto_output_log_screened, index_col=0)['base_asset']
            ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command('cat {}'.format(crypto_output_log_screened))
            tradable_pairs = pd.read_csv(StringIO(str(ssh_stdout.read().decode("utf-8"))))
            tradable_assets = tradable_pairs['symbol'].apply(lambda pair: get_base_asset_from_pair(pair, 
                                                                                                   exchange_info))
            if tradable_assets.size < 1:
                to_asset = sell_asset
            else:
                tradable_assets = list(set(tradable_assets.tolist()))
                latest_asset = tradable_assets[-1]
                if from_asset not in tradable_assets:
                    to_asset = latest_asset

            if from_asset != to_asset:
                conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
                #conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
                from_asset, converted_quantity, quantity = \
                    select_asset_with_biggest_wallet(client=client, conversion_table=conversion_table, 
                                                     exchange_info=exchange_info)
                request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                                exchange_info=exchange_info, priority='wallet')
                if request is not None:
                    if to_asset != sell_asset:
                        pair_bought = request['symbol']
                        price_when_bought = float(request['fills'][0]['price'])
                    else:
                        asset_to_delete = from_asset
                    from_asset = to_asset
            elif bought_asset is not None:
                conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
                #conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
                price_now = conversion_table[conversion_table['symbol'] == bought_asset]['lastPrice'].iat[0]
                percent_gain = ((price_now - price_when_bought) / price_when_bought) * 100.0
                if stop_loss is not None:
                    if percent_gain <= -stop_loss:
                        to_asset = latest_asset
                elif take_profit is not None:
                    if percent_gain >= take_profit:
                        to_asset = latest_asset
                else:
                    asset_to_delete = None
            time.sleep(2)
        else:
            input_pairs = pd.read_csv(crypto_input_log_screened, index_col=0)
            input_pairs = input_pairs[input_pairs['base_asset'] != asset_to_delete]
            input_pairs.to_csv(crypto_input_log_screened)
            asset_to_delete = None
    except (IndexError, pd.errors.EmptyDataError):
        time.sleep(0.5)
        continue

ssh.close()

In [ ]:
take_profit = None
stop_loss = None
sell_asset = 'BTC'
bought_asset = None
asset_to_delete = None
keys_file = 'contabo_keys.txt'
crypto_input_log_screened = 'crypto_logs/crypto_input_log_15s_screened.txt'
crypto_output_log_screened = '~/workspace/crypto_logs/crypto_output_log_1min_screened.txt'

with open(keys_file, 'r') as f:
    password = f.readline().replace('\n', '')

from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.conversion import get_base_asset_from_pair
from cryptocurrency.exchange import Cryptocurrency_exchange

import sys
import time
import pandas as pd
import paramiko

if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect('154.12.239.24', username='sam', port=22, password=password)

client = Cryptocurrency_authenticator(use_keys=False, testnet=False).spot_client
exchange_info = Cryptocurrency_exchange(client=client, directory='crypto_logs').info

conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
#conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
from_asset, converted_quantity, quantity = \
    select_asset_with_biggest_wallet(client=client, conversion_table=conversion_table, 
                                     exchange_info=exchange_info)

while True:
    try:
        if asset_to_delete is None:
            #tradable_assets = pd.read_csv(crypto_output_log_screened, index_col=0)['base_asset']
            ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command('cat {}'.format(crypto_output_log_screened))
            tradable_pairs = pd.read_csv(StringIO(str(ssh_stdout.read().decode("utf-8"))))
            tradable_assets = tradable_pairs['symbol'].apply(lambda pair: get_base_asset_from_pair(pair, 
                                                                                                   exchange_info))
            if tradable_assets.size < 1:
                to_asset = sell_asset
            else:
                tradable_assets = list(set(tradable_assets.tolist()))
                latest_asset = tradable_assets[-1]
                if from_asset not in tradable_assets:
                    to_asset = latest_asset

            if from_asset != to_asset:
                conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
                #conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
                from_asset, converted_quantity, quantity = \
                    select_asset_with_biggest_wallet(client=client, conversion_table=conversion_table, 
                                                     exchange_info=exchange_info)
                request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                                exchange_info=exchange_info, priority='wallet')
                if request is not None:
                    if to_asset != sell_asset:
                        pair_bought = request['symbol']
                        price_when_bought = float(request['fills'][0]['price'])
                    else:
                        asset_to_delete = from_asset
                    from_asset = to_asset
            elif bought_asset is not None:
                conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
                #conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
                price_now = conversion_table[conversion_table['symbol'] == bought_asset]['lastPrice'].iloc[0]
                percent_gain = ((price_now - price_when_bought) / price_when_bought) * 100.0
                if stop_loss is not None:
                    if percent_gain <= -stop_loss:
                        to_asset = latest_asset
                elif take_profit is not None:
                    if percent_gain >= take_profit:
                        to_asset = latest_asset
                else:
                    asset_to_delete = None
            time.sleep(2)
        else:
            input_pairs = pd.read_csv(crypto_input_log_screened, index_col=0)
            input_pairs = input_pairs[input_pairs['base_asset'] != asset_to_delete]
            input_pairs.to_csv(crypto_input_log_screened)
            asset_to_delete = None
    except (IndexError, pd.errors.EmptyDataError):
        time.sleep(0.5)
        continue

ssh.close()

In [ ]:
'''
take_profit = 10.0
stop_loss = None
sell_asset = 'BTC'
bought_asset = None
asset_to_delete = None
crypto_input_log_screened = 'crypto_logs/crypto_input_log_1min_screened.txt'
crypto_output_log_screened = 'crypto_logs/crypto_output_log_30min_screened.txt'

from time import sleep
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange

client = Cryptocurrency_authenticator(use_keys=False, testnet=False).spot_client
exchange_info = Cryptocurrency_exchange(client=client, directory='crypto_logs').info

#conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
from_asset, converted_quantity, quantity = \
    select_asset_with_biggest_wallet(client=client, conversion_table=conversion_table, 
                                     exchange_info=exchange_info)

while True:
    try:
        if asset_to_delete is None:
            tradable_assets = pd.read_csv(crypto_output_log_screened, index_col=0)['base_asset']
            if tradable_assets.size < 1:
                to_asset = sell_asset
            else:
                tradable_assets = list(set(tradable_assets.tolist()))
                latest_asset = tradable_assets[-1]
                if from_asset not in tradable_assets:
                    to_asset = latest_asset

            if from_asset != to_asset:
                #conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
                conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
                request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                                exchange_info=exchange_info, priority='wallet')
                if request is not None:
                    if to_asset != sell_asset:
                        pair_bought = request['symbol']
                        price_when_bought = float(request['fills'][0]['price'])
                    else:
                        asset_to_delete = from_asset
                    from_asset = to_asset
            elif bought_asset is not None:
                #conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
                conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
                price_now = conversion_table[conversion_table['symbol'] == bought_asset]['lastPrice'].iloc[0]
                percent_gain = ((price_now - price_when_bought) / price_when_bought) * 100.0
                if stop_loss is not None:
                    if percent_gain <= -stop_loss:
                        to_asset = latest_asset
                elif take_profit is not None:
                    if percent_gain >= take_profit:
                        to_asset = latest_asset
                else:
                    asset_to_delete = None
            sleep(5)
        else:
            input_pairs = pd.read_csv(crypto_input_log_screened, index_col=0)
            input_pairs = input_pairs[input_pairs['base_asset'] != asset_to_delete]
            input_pairs.to_csv(crypto_input_log_screened)
            asset_to_delete = None
    except (IndexError, pd.errors.EmptyDataError):
        sleep(0.5)
        continue
'''

In [ ]:
keys_file = 'contabo_keys.txt'
output_log_screened = '~/workspace/crypto_logs/crypto_output_log_1min_screened.txt'

with open(keys_file, 'r') as f:
    password = f.readline().replace('\n', '')

import sys
import time
import pandas as pd
import paramiko

if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect('154.12.239.24', username='sam', port=22, password=password)

while True:
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command('cat {}'.format(crypto_output_log_screened))

    df = pd.read_csv(StringIO(str(ssh_stdout.read().decode("utf-8"))))

    if df.shape[0] < 1:
        print('None')
    else:
        print(df)

    time.sleep(5)

ssh.close()

In [ ]:
try:
    tradable_pairs = pd.read_csv(crypto_output_log_screened, index_col=0)['symbol']
    conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
    tradable_assets = [get_base_asset_from_pair(tradable_pair, exchange_info=exchange_info) 
                       for tradable_pair in tradable_pairs.tolist()]
    tradable_assets = list(set(tradable_assets))
except pd.errors.EmptyDataError:
    sleep(0.5)
    tradable_assets = []
tradable_assets

In [ ]:
def get_highest_daily_volume_pair_associated_with_base_asset(base_asset, conversion_table, exchange_info):
    pairs = exchange_info[exchange_info['baseAsset'] == base_asset]['symbol'].tolist()
    filtered_pairs = conversion_table[conversion_table['symbol'].isin(pairs)]
    filtered_pairs = filtered_pairs.sort_values(by=['volume'], ascending=False)
    return filtered_pairs['symbol'].iloc[0]

pair = get_highest_daily_volume_pair_associated_with_base_asset(base_asset='LSK', 
                                                                conversion_table=conversion_table, 
                                                                exchange_info=exchange_info)

pair

In [ ]:
def get_highest_daily_volume_pair_associated_with_base_asset(base_asset, conversion_table, exchange_info):
    pairs = exchange_info[exchange_info['baseAsset'] == base_asset]['symbol'].tolist()
    filtered_pairs = conversion_table[conversion_table['symbol'].isin(pairs)]
    filtered_pairs = filtered_pairs.sort_values(by=['volume'], ascending=False)
    return filtered_pairs['symbol'].iloc[0]

crypto_input_log_screened_name = 'crypto_logs/crypto_input_log_1min_screened.txt'
dataset = pd.read_csv(crypto_input_log_screened_name, index_col=0)

dataset['base_asset'] = dataset['symbol'].apply(lambda x: get_base_asset_from_pair(x, exchange_info=exchange_info))
dataset['symbol'] = dataset['base_asset'].apply(lambda x: get_highest_daily_volume_pair_associated_with_base_asset(
                                                              base_asset=x, 
                                                              conversion_table=conversion_table, 
                                                              exchange_info=exchange_info
                                                          ))
dataset = dataset.drop_duplicates(subset=['symbol'], keep='last')
dataset